In [11]:
## ref : https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/optimize-llama-2-gptq.ipynb

In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
!sudo -H pip install auto-gptq --no-cache-dir

     |████████████████████████████████| 4.8MB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 266kB 109.0MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 113.8MB/s eta 0:00:01


  Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [3]:
!sudo -H pip install --upgrade optimum

     |████████████████████████████████| 409kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 92kB 10.5MB/s eta 0:00:01


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

In [2]:
model_name = "/data/quantization-trials/merged-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

In [4]:
quantizer = GPTQQuantizer(bits=4, dataset="wikitext2")
quantizer.quant_method = "gptq"

In [ ]:
quantized_model = quantizer.quantize_model(model, tokenizer)

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/24 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# save the quantize model to disk
quant_path = "/data/quantization-trials/GPTQ-quantized"
quantized_model.save_pretrained(save_folder, safe_serialization=True)